In [8]:
archivoVentas = 'ventasPrueba.csv'

import pandas as pd
import numpy as np

from IPython.display import display
pd.options.display.max_columns = None

pd.set_option("display.max_rows",1000)

venta_df = pd.read_csv(archivoVentas)
venta_df.head()

,idApunte,IdClinica,Nombre_Clinica,idContacto,nombre,apellidos,dni,Doctor,fechaApunte,Mes,Id_Tratamiento,Nombre_Tratamiento,Pieza,Cantidad,Precio,Precio_sin_descuento,Descuento_Porcentaje,Vr_descontado,Precio_con_descuento,TotalPagos
0,37436984,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,01/02/2018 8:49,February,16711,MENSUALIDAD ORTODONCIA METALICA,11,6,70000,1260000,17.9,225540,1034460,50000
1,37771381,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,MARIA APONTE MUNAR,01/02/2018 10:22,February,16711,MENSUALIDAD ORTODONCIA METALICA,11,3,70000,1260000,17.9,225540,1034460,16000
2,38147237,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,01/02/2018 9:44,February,16722,RETENEDOR FIJO 3X3 EN ALAMBRE,11,1,150000,150000,17.9,26850,123150,12650
3,38147242,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,01/02/2018 9:44,February,16721,RETENEDOR ESSIX O ESTETICOS,11,1,150000,150000,17.9,26850,123150,123150
4,38147247,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,01/02/2018 9:44,February,16723,RETIRO BRACKETS ARCADA INFERIOR,31,1,100000,100000,17.9,17900,82100,82100


In [6]:
venta_df = venta_df.dropna()

venta_df['ttmto_homol']=venta_df['Nombre_Tratamiento']

#diccionario de homologacion para VENTA y homologacion (si no esta incluida una homol, queda Nombre_Tratamiento)
di = {"RASPAJE Y PULIDO CORONAL(DETRARTAJE SUPRAGINGIVAL) X CUADRANTE": "RASPAJE_CUADRANTE", 
      "RETENEDOR FIJO 3X3 EN ALAMBRE": "RETENEDOR FIJO","RETENEDOR ESSIX O ESTETICOS": "RETENEDOR ESSIX", 
      "RETIRO BRACKETS ARCADA INFERIOR": "RETIRO BRACKETS ARCADA","RETIRO BRACKETS ARCADA SUPERIOR": "RETIRO BRACKETS ARCADA"}
venta_df['ttmto_homol'].replace(di, inplace=True)


#Recorte de Fechas
venta_df['fechaApunte'] = venta_df['fechaApunte'].str[0:10]

# CAMBIAR FORMATO DE COLUMNA 'FECHA CITA' A DATE (SIN HORA) DE PROD
temp_series = pd.Series() 

temp_series = venta_df['fechaApunte']
venta_df['fechaApunte'] = pd.to_datetime(temp_series)

# concat nombre y apellido en VENTAS
venta_df['Paciente'] = venta_df['nombre'] + ' ' + venta_df['apellidos']


# Resumen de CANTIDADES en Dataframe de VENTA
ventaResumen = venta_df.groupby(['Nombre_Clinica','Paciente','dni','ttmto_homol','fechaApunte'], as_index = False).sum()
ventaResumen['fecha_prog'] = '' #esta fecha se llena en clinica y se va corriendo auto a medida que se evoluciona, en el cruce
ventaResumen['Semana_prog']=None
ventaResumen['Semana_prod']=0 
ventaResumen['Fecha_vigencia']=None
ventaResumen['Cant_prod']=0
ventaResumen['Cant_pend']=ventaResumen['Cantidad'] - ventaResumen['Cant_prod']

ventaResumen = ventaResumen[['Nombre_Clinica','Paciente','dni','ttmto_homol','Cantidad','Cant_prod','Cant_pend','fechaApunte','Semana_prog','Fecha_vigencia']]
ventaResumen['fechaApunte'] = pd.to_datetime(ventaResumen['fechaApunte'].dt.strftime('%d/%m/%Y'))
ventaResumen = ventaResumen.rename(index=str, columns={"fechaApunte": "fechaVenta"})

ventaResumen.to_csv('progNueva1.csv', index = False)
print("Archivo generado! Ingresar semanas de programacion")
ventaResumen

Archivo generado! Ingresar semanas de programacion


,Nombre_Clinica,Paciente,dni,ttmto_homol,Cantidad,Cant_prod,Cant_pend,fechaVenta,Semana_prog,Fecha_vigencia
0,Autopista 104A Bogota,EVELYN DAZA,111,MENSUALIDAD ORTODONCIA METALICA,9,0,9,2018-02-01,None,None
1,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR ESSIX,1,0,1,2018-02-01,None,None
2,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR FIJO,2,0,2,2018-02-01,None,None
3,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,2,0,2,2018-02-01,None,None
4,Autopista 104A Bogota,PEDRO GOMEZ,222,DETARTRAJE POR ARCADA,2,0,2,2018-03-01,None,None
5,Autopista 104A Bogota,PEDRO GOMEZ,222,PROFILAXIS,2,0,2,2018-03-01,None,None
